In [7]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [20]:
import torch
import torchvision
from glob import glob
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transform
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import torch
import torchvision.models as models
import torchvision.transforms as transforms

In [13]:
train_path = glob("./cityscapes_data/train/*")
valid_path = glob("./cityscapes_data/val/*")

In [ ]:
fig,ax = plt.subplots(5,2,figsize=(10,30))
for i in range(5):
    img = plt.imread(train_path[i])
    ax[i][0].imshow(img[:,:256])
    ax[i][1].imshow(img[:,256:])

In [ ]:
train_dataset = []
validation_dataset = []

In [ ]:
# class MyDataset(Dataset):

#     def __init__(self, images_path, transform_img=None, transform_label=None):

#         self.images_path = images_path
#         self.transform_img = transform_img
#         self.transform_label = transform_label

#     def __len__(self):
#         return len(self.images_path)

#     def __getitem__(self, idx):

#         img = plt.imread(self.images_path[idx])
#         image, label = img[:, : int(img.shape[1] / 2)], img[:, int(img.shape[1] / 2) :]

#         if self.transform_img:
#             image = self.transform_img(image)

#         if self.transform_label:
#             label = self.transform_label(label)

#         return image, label

In [30]:
mytransforms = transform.Compose(
    [
        
        transform.ToTensor(),
        transform.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

traindata = MyDataset(train_path, mytransforms, mytransforms)
valdata = MyDataset(valid_path, mytransforms, mytransforms)

train_loader = torch.utils.data.DataLoader(traindata, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(valdata, batch_size=32, shuffle=False)

In [29]:
from PIL import Image


class MyDataset(Dataset):
    def __init__(self, images_path, transform_img=None, transform_label=None):
        self.images_path = images_path
        self.transform_img = transform.Compose([transform.ToTensor(), transform_img])
        self.transform_label = transform_label

    def __len__(self):
        return len(self.images_path)

    def __getitem__(self, idx):
        img = plt.imread(self.images_path[idx])
        image, label = img[:, :256], img[:, 256:]

        # Convert image and label from NumPy arrays to PIL Images
        image = Image.fromarray(image)
        label = Image.fromarray(label)

        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_label:
            label = self.transform_label(label)

        return image, label

In [9]:
# mytransforms = transform.Compose(
#     [
#         transform.ToTensor(),
#         transform.RandomHorizontalFlip(p=0.9),
#     ]
# )

In [31]:
import torch
from torchvision import models, transforms
from tqdm import tqdm


def extract_features(data_loader, model):
    model.eval()
    features = []

    with torch.no_grad():
        for images, _ in tqdm(data_loader):
            images = images.to(device)
            batch_features = model.features(images)
            batch_features = batch_features.view(batch_features.size(0), -1)
            features.append(batch_features.cpu().numpy())

    features = np.concatenate(features, axis=0)
    return features


# Preprocessing transformations
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Load the pre-trained VGG19 model
vgg19 = models.vgg19(pretrained=True)
vgg19 = vgg19.to(device)

# Extract features from the train and validation datasets
train_features = extract_features(train_loader, vgg19)
val_features = extract_features(val_loader, vgg19)

  0%|          | 0/93 [00:00<?, ?it/s]


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [14]:
mytransforms = transform.Compose([transform.RandomHorizontalFlip(p=0.9)])

traindata = MyDataset(train_path, mytransforms, mytransforms)
valdata = MyDataset(valid_path, mytransforms, mytransforms)

In [18]:
train_loader = torch.utils.data.DataLoader(traindata, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(valdata, batch_size=32, shuffle=False)

In [19]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

In [21]:
vgg19 = models.vgg19(pretrained=True)

/home/voy/.conda/envs/chinaa312/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/voy/.conda/envs/chinaa312/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
(train_loader)

AttributeError: 'DataLoader' object has no attribute 'shape'

In [34]:
def extract_features(dataloader):
    features = []

    with torch.no_grad():
        for batch in dataloader:
            batch_features = vgg19.features(batch)
            batch_features = batch_features.view(batch_features.size(0), -1)
            features.append(batch_features.numpy())

    features = np.concatenate(features, axis=0)
    return features


train_features = extract_features(train_loader)
val_features = extract_features(val_loader)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [ ]:
# train dataset
traindata = MyDataset(train_path, mytransforms, mytransforms)
# val dataset
valdata = MyDataset(valid_path, mytransforms, mytransforms)

In [ ]:
# Creating the DataLoaders
batch_size = 4
train_loader = DataLoader(traindata, batch_size)
vaild_loader = DataLoader(valdata, 1)

In [ ]:
def show(img, output, label, denorm=False):
    img, output, label = img.cpu(), output.cpu(), label.cpu()
    fig, ax = plt.subplots(len(output), 3, figsize=(10, 10))

    for i in range(len(output)):
        if len(output) == 3:
            Img, Lab, act = img[i], output[i], label[i]
            Img, Lab, act = Img, Lab.detach().permute(1, 2, 0).numpy(), act
            ax[i][0].imshow(Img.permute(1, 2, 0))
            ax[i][1].imshow(Lab)
            ax[i][2].imshow(act.permute(1, 2, 0))
        else:
            Img, Lab, act = img[i], output[i], label[i]
            Img, Lab, act = Img, Lab.detach().permute(1, 2, 0).numpy(), act
            ax[0].imshow(Img.permute(1, 2, 0))
            ax[1].imshow(Lab)
            ax[2].imshow(act.permute(1, 2, 0))
    plt.show()

In [ ]:
def show(img, output, label, denorm=False):
    img, output, label = img.cpu(), output.cpu(), label.cpu()
    fig, ax = plt.subplots(len(output), 3, figsize=(10, 10))
    for i in range(len(output)):
        if len(output) == 3:
            Img, Lab, act = img[i], output[i], label[i]
            Img = Img.permute(1, 2, 0).numpy()
            Lab = Lab.detach().permute(1, 2, 0).numpy()
            act = act.permute(1, 2, 0).numpy()
            ax[i][0].imshow(Img)
            ax[i][1].imshow(Lab)
            ax[i][2].imshow(act)
        else:
            Img, Lab, act = img[i], output[i], label[i]
            Img = Img.permute(1, 2, 0).numpy()
            Lab = Lab.detach().permute(1, 2, 0).numpy()
            act = act.permute(1, 2, 0).numpy()
            ax[0].imshow(Img)
            ax[1].imshow(Lab)
            ax[2].imshow(act)
    plt.show()

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


class Convblock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.LeakyReLU(0.01),
            nn.Dropout(0.1),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.LeakyReLU(0.01),
        )

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, input_channel, retain=True):
        super().__init__()
        self.conv1 = Convblock(input_channel, 32)
        self.conv2 = Convblock(32, 64)
        self.conv3 = Convblock(64, 128)
        self.conv4 = Convblock(128, 256)
        self.neck = nn.Conv2d(256, 512, 3, 1, 1)
        self.upconv4 = nn.ConvTranspose2d(512, 256, 3, 2, 1, 1)
        self.dconv4 = Convblock(512, 256)
        self.upconv3 = nn.ConvTranspose2d(256, 128, 3, 2, 1, 1)
        self.dconv3 = Convblock(256, 128)
        self.upconv2 = nn.ConvTranspose2d(128, 64, 3, 2, 1, 1)
        self.dconv2 = Convblock(128, 64)
        self.upconv1 = nn.ConvTranspose2d(64, 32, 3, 2, 1, 1)
        self.dconv1 = Convblock(64, 32)
        self.out = nn.Conv2d(32, 3, 1, 1)
        self.retain = retain

    def forward(self, x):
        # Encoder Network
        # Conv down 1
        conv1 = self.conv1(x)
        pool1 = F.max_pool2d(conv1, kernel_size=2, stride=2)

        # Conv down 2
        conv2 = self.conv2(pool1)
        pool2 = F.max_pool2d(conv2, kernel_size=2, stride=2)

        # Conv down 3
        conv3 = self.conv3(pool2)
        pool3 = F.max_pool2d(conv3, kernel_size=2, stride=2)

        # Conv down 4
        conv4 = self.conv4(pool3)
        pool4 = F.max_pool2d(conv4, kernel_size=2, stride=2)

        # Bottleneck
        neck = self.neck(pool4)

        # Decoder Network
        # Upconv 1
        upconv4 = self.upconv4(neck)
        cropped = self.crop(conv4, upconv4)
        # Making the skip connection 1
        dconv4 = self.dconv4(torch.cat([upconv4, cropped], 1))

        # Upconv 2
        upconv3 = self.upconv3(dconv4)
        cropped = self.crop(conv3, upconv3)
        # Making the skip connection 2
        dconv3 = self.dconv3(torch.cat([upconv3, cropped], 1))

        # Upconv 3
        upconv2 = self.upconv2(dconv3)
        cropped = self.crop(conv2, upconv2)
        # Making the skip connection 3
        dconv2 = self.dconv2(torch.cat([upconv2, cropped], 1))

        # Upconv 4
        upconv1 = self.upconv1(dconv2)
        cropped = self.crop(conv1, upconv1)
        # Making the skip connection 4
        dconv1 = self.dconv1(torch.cat([upconv1, cropped], 1))

        # Output Layer
        out = self.out(dconv1)
        if self.retain:
            out = F.interpolate(out, list(x.shape)[2:])

        return out

    def crop(self, input_tensor, target_tensor):
        # For making the size of the encoder conv layer and the decoder Conv layer same
        _, _, H, W = target_tensor.shape
        return transforms.CenterCrop([H, W])(input_tensor)

In [ ]:
model = UNet(3).float().to(device)

In [ ]:
lr = 0.01
epochs = 300

In [ ]:
# Choosing the loss function to be Mean Square Error Loss
lossfunc = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_acc = []
val_acc = []
train_loss = []
val_loss = []

In [ ]:
# for img , label in tqdm(train_loader):
#     print(img.shape)

In [ ]:
'''
    Traning the Model.
# '''
# img , label  = train_loader
# optimizer.zero_grad()
# img = img.to(device)
# label = label.to(device)
# output = model(img)
# loss = lossfunc(output,label)
# loss.backward()
# optimizer.step()
# trainloss+=loss.item()
# show(img, output, label)

In [ ]:
for i in range(epochs):
    
    trainloss = 0
    valloss = 0
    
    for img,label in tqdm(train_loader):
        '''
            Traning the Model.
        '''
        optimizer.zero_grad()
        img = img.to(device)
        label = label.to(device)
        output = model(img)
        loss = lossfunc(output,label)
        loss.backward()
        optimizer.step()
        trainloss+=loss.item()
    
    if(i%5==0):
        show(img,output,label)

    train_loss.append(trainloss/len(train_loader))    
  
    for img,label in tqdm(vaild_loader):
        '''
            Validation of Model.
        '''
        img = img.to(device)
        label = label.to(device)
        output = model(img)
        loss = lossfunc(output,label)
        valloss+=loss.item()
        
    val_loss.append(valloss/len(vaild_loader))  
    
    print("epoch : {} ,train loss : {} ,valid loss : {} ".format(i,train_loss[-1],val_loss[-1]))

In [ ]:
from torchsummary import summary

In [ ]:
# Get a sample image and label from the dataset
image, label = traindata[0]

# Print the size of the image tensor
print("Input image size:", label.size())

In [ ]:
summary(model, image.size())